In [2]:
# -*- coding: utf-8 -*-
"""
Created on Sun Apr 19 17:11:57 2020

@author: wuzix
"""


import pandas as pd
import numpy as np
import keras
from keras import models
from keras import layers
from keras import Input
from keras.models import load_model
from keras.layers.merge import concatenate
from keras.optimizers import RMSprop
import os


def generator(data, lookback, step, min_index, max_index, batch_size):
        i=min_index+lookback
        while 1:
            if i+batch_size>=max_index:
                i=min_index+lookback
            rows=np.arange(i, min(i+batch_size, max_index), step)
            np.random.shuffle(rows)
            i += batch_size
            
            samples1=np.zeros((len(rows), lookback, 12))
            samples2=np.zeros((len(rows), lookback, 82))
            targets=np.zeros((len(rows), ))
            for j, row in enumerate(rows):
                indices=range(row-lookback, row)
                samples1[j]=data[indices,:12]
                samples2[j]=data[indices,12:data.shape[1]-1]
                targets[j]=data[row][data.shape[1]-1]
            yield {'num':samples1,'text':samples2}, targets



model_path='./model/zx_event_0.01_删0.h5'
#data_dir = '../建立财务矩阵/threshold/afterMerge/event'
data_dir = '../建立财务矩阵/threshold/afterMerge'
ths=[0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09]

for th in ths:
    print(th)
    fname =os.path.join(data_dir,'Zhongxing财务_删0_label'+str(th)+'_forVisual_afterMerge.csv')
    data=pd.read_csv(fname,encoding='utf-8')
    
    num=len(data)-1 #the first column is title
    train_size=1297
    #test_size=num-train_size
    # 测试数据的长度
    train_data=data.iloc[1:train_size,1:].fillna(0)
    train_data=np.array(train_data,dtype=float)
    train_data=np.asarray(train_data).astype('float32')
    
    test_data=data.iloc[train_size:,1:].fillna(0)
    test_data=np.array(test_data,dtype=float)
    test_data=np.asarray(test_data).astype('float32')
    
    #lookback=12
    #step=14
    #batch_size=140
    
    lookback=12
    step=14
    batch_size=140
    bound=bound=train_size*3//4
    callbacks_list=[
               keras.callbacks.EarlyStopping(monitor='acc', patience=1,),
               keras.callbacks.ModelCheckpoint(filepath=model_path, 
                                             monitor='val_loss', save_best_only=True,)]
    
    
    train_gen=generator(train_data, lookback=lookback, step=step, min_index=0, max_index=bound, batch_size=batch_size)
    val_gen=generator(train_data, lookback=lookback, step=step, min_index=bound+1, max_index=len(train_data), batch_size=batch_size)
    test_gen=generator(test_data, lookback=lookback, step=step, min_index=0, max_index=len(test_data), batch_size=98)
    #all_gen=generator(all_data, lookback=lookback, step=step, min_index=0, max_index=len(all_data), batch_size=batch_size)
            
    train_steps=(bound-lookback)//batch_size
    val_steps=(len(train_data)-bound+1-lookback)//batch_size
    test_steps=(len(test_data)-lookback)//98
    #all_steps=(len(all_data)-lookback)//batch_size
    
    num_input=Input(shape=(None,12),dtype='float32',name='num')
    encoded_num=layers.LSTM(64)(num_input)
    text_input=Input(shape=(None,82),dtype='float32',name='text')
    encoded_text=layers.LSTM(128)(text_input)
    concatenated=layers.concatenate([encoded_num,encoded_text],axis=-1)
    concatenated=layers.Dense(128,activation='relu')(concatenated)
    
    
    
    preds=layers.Dense(1,activation='sigmoid')(concatenated)
    model=models.Model([num_input,text_input],preds)
    model.compile(optimizer=RMSprop(),metrics=['acc'],loss='binary_crossentropy')
    history=model.fit_generator(train_gen, steps_per_epoch=train_steps, epochs=20, 
                                    validation_data=val_gen, validation_steps=val_steps,
                                    callbacks=callbacks_list, verbose=0)
    
    model=models.load_model(model_path)
    results1=model.evaluate_generator(train_gen, steps=train_steps)
    results2=model.evaluate_generator(val_gen, steps=val_steps)
    results3=model.evaluate_generator(test_gen,steps=test_steps)
    print (model.metrics_names)
    print('event', th, lookback)
    print(results1)
    print(results2)
    print(results3)
    
    
    preds2=model.predict_generator(test_gen, steps=test_steps)[:,0]
    targets2=[]
    


0.01
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



ValueError: could not broadcast input array from shape (12,0) into shape (12,82)

In [3]:
# -*- coding: utf-8 -*-
"""
Created on Sun Apr 19 17:11:57 2020

@author: wuzix
"""


import pandas as pd
import numpy as np
import keras
from keras import models
from keras import layers
from keras import Input
from keras.models import load_model
from keras.layers.merge import concatenate
from keras.optimizers import RMSprop
import os


def generator(data, lookback, step, min_index, max_index, batch_size):
        i=min_index+lookback
        while 1:
            if i+batch_size>=max_index:
                i=min_index+lookback
            rows=np.arange(i, min(i+batch_size, max_index), step)
            np.random.shuffle(rows)
            i += batch_size
            
            samples1=np.zeros((len(rows), lookback, 12))
            samples2=np.zeros((len(rows), lookback, 82))
            targets=np.zeros((len(rows), ))
            for j, row in enumerate(rows):
                indices=range(row-lookback, row)
                samples1[j]=data[indices,:12]
                samples2[j]=data[indices,12:data.shape[1]-1]
                targets[j]=data[row][data.shape[1]-1]
            yield {'num':samples1,'text':samples2}, targets



In [4]:
model_path='./model/zx_event_0.01_删0.h5'
#data_dir = '../建立财务矩阵/threshold/afterMerge/event'
data_dir = '../建立财务矩阵/threshold/afterMerge'
ths=[0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09]

for th in ths:
    print(th)
    fname =os.path.join(data_dir,'Zhongxing财务_删0_label'+str(th)+'_forVisual_afterMerge.csv')
    data=pd.read_csv(fname,encoding='utf-8')
    
    num=len(data)-1 #the first column is title
    train_size=1297
    #test_size=num-train_size
    # 测试数据的长度
    train_data=data.iloc[1:train_size,1:].fillna(0)
    train_data=np.array(train_data,dtype=float)
    train_data=np.asarray(train_data).astype('float32')
    
    test_data=data.iloc[train_size:,1:].fillna(0)
    test_data=np.array(test_data,dtype=float)
    test_data=np.asarray(test_data).astype('float32')
    
    #lookback=12
    #step=14
    #batch_size=140
    
    lookback=12
    step=14
    batch_size=140
    bound=bound=train_size*3//4
    callbacks_list=[
               keras.callbacks.EarlyStopping(monitor='acc', patience=1,),
               keras.callbacks.ModelCheckpoint(filepath=model_path, 
                                             monitor='val_loss', save_best_only=True,)]
    
    
    train_gen=generator(train_data, lookback=lookback, step=step, min_index=0, max_index=bound, batch_size=batch_size)
    val_gen=generator(train_data, lookback=lookback, step=step, min_index=bound+1, max_index=len(train_data), batch_size=batch_size)
    test_gen=generator(test_data, lookback=lookback, step=step, min_index=0, max_index=len(test_data), batch_size=98)
    #all_gen=generator(all_data, lookback=lookback, step=step, min_index=0, max_index=len(all_data), batch_size=batch_size)
            
    train_steps=(bound-lookback)//batch_size
    val_steps=(len(train_data)-bound+1-lookback)//batch_size
    test_steps=(len(test_data)-lookback)//98
    #all_steps=(len(all_data)-lookback)//batch_size
    
    num_input=Input(shape=(None,12),dtype='float32',name='num')
    encoded_num=layers.LSTM(64)(num_input)
    text_input=Input(shape=(None,82),dtype='float32',name='text')
    encoded_text=layers.LSTM(128)(text_input)
    concatenated=layers.concatenate([encoded_num,encoded_text],axis=-1)
    concatenated=layers.Dense(128,activation='relu')(concatenated)
    
    
    
    preds=layers.Dense(1,activation='sigmoid')(concatenated)
    model=models.Model([num_input,text_input],preds)
    model.compile(optimizer=RMSprop(),metrics=['acc'],loss='binary_crossentropy')
    history=model.fit_generator(train_gen, steps_per_epoch=train_steps, epochs=20, 
                                    validation_data=val_gen, validation_steps=val_steps,
                                    callbacks=callbacks_list, verbose=0)
    
    model=models.load_model(model_path)
    results1=model.evaluate_generator(train_gen, steps=train_steps)
    results2=model.evaluate_generator(val_gen, steps=val_steps)
    results3=model.evaluate_generator(test_gen,steps=test_steps)
    print (model.metrics_names)
    print('event', th, lookback)
    print(results1)
    print(results2)
    print(results3)
    
    
    preds2=model.predict_generator(test_gen, steps=test_steps)[:,0]
    targets2=[]
    


0.01


ValueError: could not broadcast input array from shape (12,0) into shape (12,82)

In [7]:
indices=range(row-lookback, row)
data[indices,:12]

NameError: name 'row' is not defined

In [10]:
i,j=1,1
rows=np.arange(i,2, step)
np.random.shuffle(rows)
samples2=np.zeros((len(rows), lookback, 82))
samples2[j]=data[indices,12:data.shape[1]-1]


NameError: name 'indices' is not defined

In [19]:
indices=range(0,12)
def generator(data, lookback, step, min_index, max_index, batch_size):
        i=min_index+lookback
        while 1:
            if i+batch_size>=max_index:
                i=min_index+lookback
            rows=np.arange(i, min(i+batch_size, max_index), step)
            np.random.shuffle(rows)
            i += batch_size
            
            samples1=np.zeros((len(rows), lookback, 12))
            samples2=np.zeros((len(rows), lookback, 82))
            print(samples1,samples2)
            targets=np.zeros((len(rows), ))
            for j, row in enumerate(rows):
                indices=range(row-lookback, row)
                samples1[j]=data[indices,:12]
                samples2[j]=data[indices,12:data.shape[1]-1]
                print(samples1[j])
                print(samples2[j])
                targets[j]=data[row][data.shape[1]-1]
            yield {'num':samples1,'text':samples2}, targets

In [20]:
model_path='./model/zx_event_0.01_删0.h5'
#data_dir = '../建立财务矩阵/threshold/afterMerge/event'
data_dir = '../建立财务矩阵/threshold/afterMerge'
th=0.01
fname =os.path.join(data_dir,'Zhongxing财务_删0_label'+str(th)+'_forVisual_afterMerge.csv')
data=pd.read_csv(fname,encoding='utf-8')
generator(data,12,14,0,972,140)


<generator object generator at 0x000001EF802DC748>

In [14]:
print(samples2[j])

IndexError: index 1 is out of bounds for axis 0 with size 1

In [22]:
data[0,12:data.shape[1]-1]

TypeError: '(0, slice(12, 13, None))' is an invalid key

In [23]:

# -*- coding: utf-8 -*-
"""
Created on Tue Mar 24 19:09:31 2020

@author: wuzix
"""
import os
import pandas as pd
import numpy as np
import keras
from keras import models
from keras import layers
#import keras.utils.Sequence
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt


def generator(data, lookback, step, min_index, max_index, batch_size):
    i=min_index+lookback
    while 1:
        if i+batch_size>=max_index:
            i=min_index+lookback
        rows=np.arange(i, min(i+batch_size, max_index), step) #把每一个batch里面的数据预测数据找出来
        i += batch_size
            
        samples=np.zeros((len(rows), lookback, 12))
        print("samples",samples)
        targets=np.zeros((len(rows), ))
        for j, row in enumerate(rows):
            indices=range(rows[j]-lookback, rows[j])
            samples[j]=data[indices,:12]   #其中 samples 是输入数据的一个批量，targets 是对应的目标温度数组
            print("samples[j]",samples[j])
            targets[j]=data[rows[j]][train_data.shape[1]-1]
        yield samples, targets


model_path='./model/zx_finance_0.01_删0.h5'
data_dir = '../建立财务矩阵/threshold/afterMerge'

#ths=[0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09]
ths=[0.01]
for th in ths:
    print(th)
    fname =os.path.join(data_dir,'Zhongxing财务_删0_label'+str(th)+'_forVisual_afterMerge.csv')
    data=pd.read_csv(fname,encoding='utf-8')
    num=len(data)-1 #the first column is title
    train_size=1297
    #test_size=num-train_size
    # 测试数据的长度
    train_data=data.iloc[1:train_size,1:].fillna(0)
    train_data=np.array(train_data,dtype=float)
    train_data=np.asarray(train_data).astype('float32')
    
    test_data=data.iloc[train_size:,1:].fillna(0)
    test_data=np.array(test_data,dtype=float)
    test_data=np.asarray(test_data).astype('float32')
    
    
    lookback=12 # 根据过去13天的数据，预测第14天的股价
    step=14  # 数据采样的周期
    batch_size=140 # 每批量的样本数
    bound=bound=len(train_data)*3//4 # 训练数据占总数量的多少
    
    
    callbacks_list=[
               keras.callbacks.EarlyStopping(monitor='acc', patience=1,),
               keras.callbacks.ModelCheckpoint(filepath=model_path, 
                                             monitor='val_loss', save_best_only=True,)]
    
    
    
    train_gen=generator(train_data, lookback=lookback, step=step, min_index=0, max_index=bound, batch_size=batch_size)
    val_gen=generator(train_data, lookback=lookback, step=step, min_index=bound+1, max_index=len(train_data), batch_size=batch_size)
    test_gen=generator(test_data, lookback=1, step=step,min_index=0, max_index=len(test_data), batch_size=batch_size)
    all_gen=generator(train_data, lookback=lookback, step=step, min_index=0, max_index=len(train_data), batch_size=batch_size)
     
    train_steps=(bound-lookback)//batch_size
    val_steps=(len(train_data)-bound+1-lookback)//batch_size
    test_steps=(len(test_data)-1)//batch_size
    
    all_steps=(len(train_data)-lookback)//batch_size
        
    # dropout LSTM
    
    model=models.Sequential()
    #model.add(layers.LSTM(64, input_shape=(None, 12)))
    model.add(layers.LSTM(64,input_shape=(None, 12)))
    model.add(layers.Dense(1,activation='sigmoid'))
    #model.compile(optimizer=RMSprop(),metrics=['acc'],loss='mae')
    model.compile(optimizer=RMSprop(),loss='binary_crossentropy',metrics=['acc'])
    history=model.fit_generator(train_gen, steps_per_epoch=train_steps, epochs=20, 
                                  validation_data=val_gen, validation_steps=val_steps,
                                   callbacks=callbacks_list, verbose=0)
    
    model=models.load_model(model_path)
    results1=model.evaluate_generator(train_gen, steps=train_steps)
    results2=model.evaluate_generator(val_gen, steps=val_steps)
    results3=model.evaluate_generator(test_gen,steps=test_steps)
    print (model.metrics_names)
    print('finance', th, lookback)
    print(results1)
    print(results2)
    print(results3)
    preds2=model.predict_generator(test_gen, steps=test_steps)[:,0]
    print(preds2)

0.01
samplessamples  [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0

samples[j] [[ 1.15191746e+00  1.12100542e+00  1.17846751e+00  1.12483418e+00
   7.86931455e-01 -1.55416131e-01 -1.54307574e-01  6.63865983e-01
  -1.09840560e+00  1.84410286e+00  2.92971402e-01  9.02972937e-01]
 [ 1.03225315e+00  1.09199750e+00  1.04250848e+00  1.13838625e+00
   2.04673386e+00  2.58648419e+00  2.21442914e+00  1.89304364e+00
  -1.10100055e+00  1.85987043e+00  2.88185060e-01  9.10552084e-01]
 [ 8.05260956e-01  9.11906600e-01  8.38569999e-01  9.42497253e-01
   1.07579517e+00 -4.31056917e-01 -3.95019442e-01  9.45675910e-01
  -1.06348014e+00  1.63244379e+00  1.65657267e-01  7.78665423e-01]
 [ 9.48364735e-01  9.43331838e-01  9.69493449e-01  9.36337233e-01
   1.97368979e-01 -6.19653225e-01 -5.54253578e-01  8.85837004e-02
  -1.06229997e+00  1.62522018e+00  1.24491505e-01  7.40057468e-01]
 [ 8.84214759e-01  9.25201893e-01  8.88925195e-01  9.89313483e-01
   9.69658017e-01  1.23729515e+00  1.05556023e+00  8.42133760e-01
  -1.07244897e+00  1.68673718e+00  1.47550821e-01  7.62540460

samples[j] [[-0.90951705 -0.93734908 -0.90497822 -0.92891985 -0.72082907 -0.22795318
  -0.4492434  -0.8016445   0.27292669 -0.99251664  0.80798024  0.48294693]
 [-0.9156853  -0.9216364  -0.89490718 -0.90920776 -0.702344    0.10571723
   0.16277857 -0.78325617  0.2810201  -0.97511786  0.84470242  0.5248341 ]
 [-0.93419009 -0.94460106 -0.91379035 -0.91783184 -0.72175503  0.06219501
   0.08296657 -0.80257004  0.27747947 -0.98272985  0.7865535   0.46979231]
 [-0.91938627 -0.92647105 -0.90371931 -0.92275983 -0.64966619 -0.18443094
  -0.36904156 -0.73080635  0.27545515 -0.98707956  0.74380231  0.4354324 ]
 [-0.8971805  -0.92284507 -0.90497822 -0.90674376 -0.48267126 -0.19893835
  -0.39168802 -0.56450945  0.28203225 -0.97294307  0.72189993  0.4269774 ]
 [-0.87127382 -0.89988047 -0.86595291 -0.88949567 -0.64095366 -0.18443094
  -0.36220154 -0.7221058   0.2891174  -0.95771915  0.83037311  0.52268767]
 [-0.8786757  -0.90229779 -0.87602395 -0.8734796  -0.70241302  0.06219501
   0.08013133 -0.7833

  -0.17259486 -0.08215598  0.2882264  -0.46892267 -0.33142707  0.2643038 ]]samples [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0

samples [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
sa

  -0.45278746 -0.54161662 -1.01200056 -0.80314976 -0.86399919 -0.98697996]]
samples[j] [[-0.77134788 -0.77417946 -0.76901919 -0.7699911  -0.61201572  0.03318019
   0.02487959 -0.6113441  -1.07820141 -0.62931752 -0.88675463 -0.9781093 ]
 [-0.70966524 -0.73671085 -0.74384159 -0.7724551  -0.49113348 -0.66317546
  -1.11758029 -0.46571863 -1.07725573 -0.63180304 -0.87084717 -0.97070795]
 [-0.70349693 -0.72341555 -0.69348645 -0.71578282 -0.67657667 -0.21344577
  -0.37715742 -0.68915498 -1.09900749 -0.57471335 -0.84075755 -0.94791299]
 [-0.67265564 -0.68352968 -0.64313126 -0.67019862 -0.5426752   0.12022464
   0.16235329 -0.5277946  -1.11650336 -0.52873093 -0.84523398 -0.93559462]
 [-0.68005753 -0.69440764 -0.67082661 -0.68005466 -0.35024673 -0.05386427
  -0.11720137 -0.29590455 -1.11272049 -0.5386731  -0.84444064 -0.93006939]
 [-0.61467391 -0.64968705 -0.65194339 -0.67512661  0.27055764 -0.82275695
  -1.30190635  0.4522154  -1.11461198 -0.53370202 -0.86889154 -0.93414766]
 [-0.46910283 -0.50

samples [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
sa

  -0.74743974  0.08475795  0.60990274 -0.31404263  0.21149825 -0.25148484]]
samples[j] [[-0.06076362 -0.07557175 -0.10433079 -0.12811597  0.47160554 -0.03935686
  -0.07811049  0.69391662  0.60990274 -0.31404263  0.15633927 -0.29076618]
 [-0.12984821 -0.1372136  -0.11188406 -0.12441996  0.3487016   0.0476876
   0.02984126  0.54588467  0.6114307  -0.31078038  0.19212522 -0.2612052 ]
 [-0.16068953 -0.14930023 -0.15216821 -0.12811597  0.63555896  0.87460989
   1.09032726  0.89143658  0.60990274 -0.31404263  0.15175194 -0.30599794]
 [-0.25321352 -0.23028071 -0.25539634 -0.20203634  0.96094728  1.57096553
   2.10225987  1.28345299  0.57929552 -0.37843359  0.07266033 -0.35317099]
 [-0.23470873 -0.21698542 -0.32085806 -0.33509299  1.03272688 -2.56364608
  -3.19715786  1.36990261  0.52420497 -0.49432176 -0.03558679 -0.44637108]
 [-0.11134341 -0.13600494 -0.11440182 -0.11702792  0.07126139  0.17825428
   0.19226508  0.21162488  0.61449069 -0.30433351  0.07600006 -0.37891456]
 [-0.16068953 -0.174

samples [[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]
samples[j] [[ 0.10454591  0.09122387  0.13359745  0.1121252  -0.75859088  0.12022464
   0.10738508 -0.78702021  0.5006907   0.07649741 -0.6029709  -0.78636849]]
samples[j] [[ 0.71273696  0.7922489   0.75296617  0.76632041 -0.49970078  0.62798399
   0.55067486 -0.33601326  0.53460795  0.17141387 -0.6810239  -0.80741888]]
samples[j] [[ 0.63008219  0.60369736  0.64722031  0.64927983 -0.86345625  0.36685058
   0.3200635  -0.87550622  0.50186312  0.07975968 -0.7377845  -0.80895776]]
samples[j] [[ 0.84473783  0.82246548  0.88263077  0.84516877 -0.87147081  0.00416537
  -0